In [39]:
#IMPORT LIBRARIES

import numpy as np
from math import floor
import tqdm.notebook as tq
from keras.datasets import mnist

In [40]:
#CREATE DATASET

#We will assume X to be an n x m matrix where m is the number of examples. Similarly y is a k x m matrix
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.array([im.flatten() for im in x_train]).T
y_train = np.array([[1 if y == i else 0 for i in range(10)] for y in y_train]).T
x_test = np.array([im.flatten() for im in x_test]).T
y_test = np.array([[1 if y == i else 0 for i in range(10)] for y in y_test]).T

In [41]:
#ACTIVATION AND LOSS FUNCTIONS AND THEIR DERIVATIVES

def relu(x):

    y = np.maximum(0, x)
    der = np.zeros_like(x)
    der[x > 0] = 1
    return y, der

#In this case each output depends on all inputs in axis 0 -> derivative is 3D matrix
def softmax(x, epsilon=0.00000001):

    n, m = x.shape
    f = []
    d = []

    for c in range(m):
        col = x[:, c]

        e = np.exp(col - col.max())
        su = e.sum(axis=0)
        y = e / su
        y = (1 - epsilon) * y + epsilon / y.shape[0] * np.ones_like(y)

        f.append(y)

        diag = np.diag(y * (1 - y))
        out = 1 - np.identity(y.shape[0])
        der = diag - out * (y[:, np.newaxis] * y[np.newaxis, :])
        d.append(der)
    
    f = np.array(f).T
    d = np.array(d)

    return f, d


def categorical_cross_entropy(y, yHat):
    
    m = y.shape[1]
    ce = - np.sum(y * np.log(yHat))
    der = - y / yHat

    return ce / m, der / m

def mse(y, yHat):
    m = y.shape[1]
    mse = np.sum((y - yHat) ** 2) / 2
    der = yHat - y
    return mse / m, der / m

In [42]:
#NEURAL NETWORK

#For the weights and biases of a layer, their dimension depends on the input and output sizes.
#Each layer also has an activation funciton
def create_dense(in_len, out_len, act):
    W = np.random.normal(0, .1, (out_len, in_len))
    b = np.random.normal(0, .1, (out_len,))
    return W, b, act

#Hidden_sizes are the output sizes of the hidden layers.
#The input size is the output size of the previous layer, or n, in the case of the first.
def create_dense_sequential(in_len, out_len, hidden_sizes, act=relu, act_final=None):

    if act_final is None:
        act_final = act

    normal_layers = [create_dense(a, b, act)
    for a, b in zip([in_len] + hidden_sizes, hidden_sizes)
    ]

    final_layer = create_dense(hidden_sizes[-1], out_len, softmax)

    return  normal_layers + [final_layer]

In [47]:
#COMPUTE THE GRADIENT WRT THE PARAMETERS

#Forward step to calculate the activations and gradients (wrt preactivations) of each layer
def forward(dense, aPrev):

    #get the parameters and activation
    W, b, act = dense

    #preactivation (linear function)
    p = W @ aPrev + b[:, np.newaxis]

    #get the activation of the layer and its derivative
    a, da_dp =  act(p)
    return (a, da_dp)


"""

We want to apply the chain rule. We are given:

    - dense: the parameters and activation funciton of the current layer.
    - dL_da: the derivative of the loss function wrt the activation of the current layer (see next on how we calculat this).
    - a_prev and da_dp: the previous layer's activation and the derivative of the current activation wrt the preactivation (both calculated with the function above).

With this, we can calculate:

    - dL_dW and dL_db: the derivatives of the loss function wrt the parameters of the current layer
    - dL_daPrev: the derivative of the loss function wrt the activation of the previous layer


"""

def backward(dense, dL_da, a_prev, da_dp):

    W, b, act = dense

    if len(da_dp.shape) == 2:
        dL_dp = dL_da * da_dp
    #In case all outputs of the activation depend on all inputs (like in softmax), in which case the derivative da_dp is a 3d matrix when we have multiple training examples
    else:
        dL_dp = np.einsum('ijk, ki -> ji', da_dp, dL_da)

    dL_dW = dL_dp @ a_prev.T
    dL_db = dL_dp.sum(axis=1)
    dL_daPrev = W.T @ dL_dp

    return (
        dL_dW,
        dL_db,
        dL_daPrev
    )

#We do the previous starting at the end (with the loss function) and we work backwards. This is called BACKPROPAGATION.
def calculate_gradients_and_loss(layers, loss, X, y, wd=0.0):

    layer_results = [(X, None)]
    for layer in layers:
        layer_results.append(forward(layer, layer_results[-1][0]))

    L, dL_da = loss(y, layer_results[-1][0])

    gradients_W = []
    gradients_b = []

    for i in reversed(range(len(layers))):
        
        a, da_dp = layer_results[i+1]
        a_prev, da_dp_prev = layer_results[i]
        layer = layers[i]
        dL_dW, dL_db, dL_da = backward(layer, dL_da, a_prev, da_dp)
        gradients_W.append(dL_dW + 2 * wd * layer[0])
        gradients_b.append(dL_db + 2 * wd * layer[1])

    return reversed(gradients_W), reversed(gradients_b), L

In [48]:
#INFERENCE AND METRICS

def get_result(layers, X):

    result = X
    for layer in layers:
        result, _ = forward(layer, result)

    return result

def acc_categorical(a, b):
    return np.mean(np.argmax(a, axis=0) == np.argmax(b, axis=0))
    

def metrics(layers, loss, X, y, X_test, y_test):

    r = get_result(layers, X)
    r_test = get_result(layers, X_test)

    out = {}

    out['loss'], _ = loss(y, r)
    out['loss_test'], _ = loss(y_test, r_test)

    out['acc'] = acc_categorical(y, r)
    out['acc_test'] = acc_categorical(y_test, r_test)

    return out

In [49]:
#TRAINING WITH GRADIENT DESCENT

def gradient_descent(layers, loss, X, y, X_test, y_test, alpha=0.01, epochs=10, batch_size=1000, wd=0.0):

    #print metrics before training
    print(metrics(layers, loss, X, y, X_test, y_test))

    #number of training examples
    m = X.shape[1]

    #in every epoch, we take these starting indices  in X for the batches
    batch_starts = range(0, m, batch_size)

    #epoch loop
    for epoch in range(epochs):

        #to avoid repetition in each epoch we shuffle the training examples
        perm = np.random.permutation(X.shape[1])
        X_rand = np.take(X, perm, axis=1)
        y_rand = np.take(y, perm, axis=1)
        
        #batch loop
        for start in tq.tqdm(batch_starts):

            #select batch_size examples
            end = min(m, start + batch_size)
            X_batch = X_rand[:, start:end]
            y_batch = y_rand[:, start:end]

            #calculate gradients wrt all parameters
            gradients_W, gradients_b, L = calculate_gradients_and_loss(layers, loss, X_batch, y_batch, wd)
            for layer, gradient_W, gradient_b in zip(layers, gradients_W, gradients_b):

                #update the values of the parameters accordingly
                W, b, _ = layer
                W -= alpha * gradient_W
                b -= alpha * gradient_b

        #print metrics at the end of each batch
        print(f'Epoch {epoch}: {metrics(layers, loss, X, y, X_test, y_test)}')

In [50]:
layers = create_dense_sequential(x_train.shape[0], y_train.shape[0], [500, 200, 100, 30, 30, 30], act=relu, act_final=softmax)
gradient_descent(layers, categorical_cross_entropy, x_train, y_train, x_test, y_test, alpha=.01, epochs=20)

{'loss': 11.941228642724152, 'loss_test': 12.0123788947772, 'acc': 0.13306666666666667, 'acc_test': 0.1377}


  0%|          | 0/60 [00:00<?, ?it/s]

Epoch 0: {'loss': 2.208176275770429, 'loss_test': 2.209161143736538, 'acc': 0.1584, 'acc_test': 0.1593}


  0%|          | 0/60 [00:00<?, ?it/s]

Epoch 1: {'loss': 1.9237427966535599, 'loss_test': 1.9118038850893102, 'acc': 0.2807, 'acc_test': 0.282}


  0%|          | 0/60 [00:00<?, ?it/s]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/ferran/PycharmProjects/machineLearning/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_34603/547338905.py", line 2, in <module>
    gradient_descent(layers, categorical_cross_entropy, x_train, y_train, x_test, y_test, alpha=.01, epochs=20)
  File "/tmp/ipykernel_34603/1418196952.py", line 40, in gradient_descent
    print(f'Epoch {epoch}: {metrics(layers, loss, X, y, X_test, y_test)}')
  File "/tmp/ipykernel_34603/3823106434.py", line 17, in metrics
    r = get_result(layers, X)
  File "/tmp/ipykernel_34603/3823106434.py", line 7, in get_result
    result, _ = forward(layer, result)
  File "/tmp/ipykernel_34603/3553233477.py", line 13, in forward
    a, da_dp =  act(p)
  File "/tmp/ipykernel_34603/730946720.py", line 21, in softmax
    su = e.sum(axis=0)
  File "/home/ferran/PycharmProjects/machineLearning/venv/lib/pyt

In [ ]:
layers = create_dense_sequential(x_train.shape[0], y_train.shape[0], [500, 200, 100, 30, 30, 30], act=relu, act_final=softmax)
gradient_descent(layers, mse, x_train, y_train, x_test, y_test, alpha=.03, epochs=20)

In [52]:
layers = create_dense_sequential(x_train.shape[0], y_train.shape[0], [500, 200, 100, 30, 30, 30], act=relu, act_final=softmax)
gradient_descent(layers, mse, x_train, y_train, x_test, y_test, alpha=.03, epochs=20, wd=0.05)

{'loss': 0.8281125194544967, 'loss_test': 0.830627020954445, 'acc': 0.09871666666666666, 'acc_test': 0.098}


  0%|          | 0/60 [00:00<?, ?it/s]

Epoch 0: {'loss': 0.3158758856389834, 'loss_test': 0.3131244675591984, 'acc': 0.4807666666666667, 'acc_test': 0.4818}


  0%|          | 0/60 [00:00<?, ?it/s]

Epoch 1: {'loss': 0.2445938273483189, 'loss_test': 0.24112879379518823, 'acc': 0.67705, 'acc_test': 0.6824}


...

Epoch 17: {'loss': 0.17680348474871388, 'loss_test': 0.17142871449806352, 'acc': 0.82575, 'acc_test': 0.8327}


  0%|          | 0/60 [00:00<?, ?it/s]

Epoch 18: {'loss': 0.17910543831182063, 'loss_test': 0.17398160256446893, 'acc': 0.8161333333333334, 'acc_test': 0.8224}


  0%|          | 0/60 [00:00<?, ?it/s]

Epoch 19: {'loss': 0.17669879902007024, 'loss_test': 0.1711967082275716, 'acc': 0.8219, 'acc_test': 0.8267}
